In [9]:
!pip install scikeras

In [1]:
# Importar bibliotecas necesarias
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


In [2]:
# Cargar los datos
data = pd.read_csv('teleCust1000t.csv')


In [3]:
# Creación de nuevas características
data['income_normalized'] = data['income'] / data['income'].max()
data['income_per_year'] = data['income'] / (data['age'] + 1e-5)
data['loyal_customer'] = (data['tenure'] > data['tenure'].mean()).astype(int)

# Normalización de características continuas
continuous_features = ['tenure', 'age', 'address', 'income', 'employ', 'income_normalized', 'income_per_year']
scaler = StandardScaler()
data[continuous_features] = scaler.fit_transform(data[continuous_features])


In [4]:
# División en variables independientes (X) y dependiente (y)
X = data.drop(columns=['custcat'])
y = data['custcat']


In [5]:
# Reducción de dimensionalidad con PCA
pca = PCA(n_components=0.95)  # Retener 95% de la varianza
X_pca = pca.fit_transform(X)


In [6]:
# División de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)


In [7]:
# Construcción del modelo de red neuronal
def build_model(learning_rate=0.001, dropout_rate=0.5):
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(4, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [11]:
# Búsqueda de hiperparámetros con Keras
from scikeras.wrappers import KerasClassifier
param_grid = {
    'model__learning_rate': [0.001, 0.0005], # Pass learning_rate to build_model
    'model__dropout_rate': [0.3, 0.5],   # Pass dropout_rate to build_model
    'batch_size': [16, 32],
    'epochs': [50, 100]
}
model = KerasClassifier(model=build_model, verbose=0)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='accuracy')
grid_result = grid.fit(X_train, y_train)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/lo

In [14]:
# Mejor modelo
best_model = grid_result.best_estimator_

# Evaluación en el conjunto de prueba
y_pred = best_model.predict(X_test)  # Get the predicted probabilities
y_pred_classes = y_pred  # Get the predicted class labels
print("Reporte de Clasificación:")
print(classification_report(y_test, y_pred_classes)) # Pass y_pred_classes to classification_report

Reporte de Clasificación:
              precision    recall  f1-score   support

           1       0.48      0.57      0.52        83
           2       0.29      0.15      0.20        65
           3       0.42      0.39      0.40        83
           4       0.38      0.51      0.44        69

    accuracy                           0.41       300
   macro avg       0.39      0.40      0.39       300
weighted avg       0.40      0.41      0.40       300



In [19]:
best_keras_model = best_model.model_
# Guardar el modelo en formato TensorFlow SavedModel
best_keras_model.save("red_neuro.keras") # Add .keras extension

# Guardar el modelo en formato HDF5
best_keras_model.save("red_neuro.h5")